In [9]:
import sympy as sy
import control as pc

### Linear Steady-State Modelling

Beamsplitter
\begin{equation}
\begin{bmatrix} E_{out0} \\ E_{out1} \end{bmatrix} =
\begin{bmatrix} r_{ac}& t_{bc} \\  t_{ad}& r_{bd} \end{bmatrix}
\begin{bmatrix} E_{in0} \\ E_{in1} \end{bmatrix}
\end{equation}

So our inputs into the control system are the electromagnetic signals, that are time-domain constrained

#### Beamsplitter
\begin{equation}
\begin{bmatrix} E_{out0}(t) \\ E_{out1}(t) \end{bmatrix} =
\begin{bmatrix} r_{ac}& t_{bc} \\  t_{ad}& r_{bd} \end{bmatrix}
\begin{bmatrix} E_{in0}(t) \\ E_{in1}(t) \end{bmatrix} = 
\begin{bmatrix} \sqrt{r} &  i\sqrt{1-r} \\  i \sqrt{1-r} & \sqrt{r} \end{bmatrix}
\begin{bmatrix} E_{in0}(t) \\ E_{in1}(t) \end{bmatrix}  =
\textbf{B}\textbf{E(t)}
\end{equation}

#### Phase Shifter on one arm

Assuming a phase shifter on top arm
\begin{equation}
\textbf{ $\Phi$ } = \begin{bmatrix} e^{i\phi} & 0 \\ 0 & 1\end{bmatrix}
\end{equation}

Note we're not yet taking into account Linear and LinearGroup delay, or the quantum probability variable within the design of the device.

Hence the MZI configuration is:

\begin{equation}
M\textbf{E(t)}  = \textbf{B} \textbf{ $\Phi$ } \textbf{B}\textbf{E(t)} 
\end{equation}

\begin{equation}
M\textbf{E(t)}  = \begin{bmatrix} \sqrt{r} &  i\sqrt{1-r} \\  i \sqrt{1-r} & \sqrt{r} \end{bmatrix} \begin{bmatrix} e^{i\phi} & 0 \\ 0 & 1\end{bmatrix} \begin{bmatrix} \sqrt{r} &  i\sqrt{1-r} \\  i \sqrt{1-r} & \sqrt{r} \end{bmatrix}\begin{bmatrix} E_{in0}(t) \\ E_{in1}(t) \end{bmatrix} = \begin{bmatrix} E_{out0}(t) \\ E_{out1}(t) \end{bmatrix} 
\end{equation}

Now we could just crunch some matrices together and input our time signals, but what if we wanted to leverage classical control system theory with regards to MIMO inputs and outputs, so that it is easier to design this type of systems togther and include feedback and nonlinearity back into the classical control models?

The standard way to start is to create the differential equations for this type of system and we can use the fast linear properties of SymPy to create a single MZI component, and the quantum probability variable could be added into this later. Right now let´s consider that we've only got differential equations, with equal reflectivity on both sides of the matrix:

In [64]:
t = sy.Symbol("t")
E_0 = sy.Function("E_{in0}")(t)
E_1 = sy.Function("E_{in1}")(t)
phi = sy.Function("\phi")(t)
r_0, r_1 = sy.symbols("r_0 r_1")
B_0  = sy.Matrix([[sy.sqrt(r_0), sy.I*sy.sqrt(1 - r_0)],
				[sy.I*sy.sqrt(1 - r_0), sy.sqrt(r_0)]])
B_1  = sy.Matrix([[sy.sqrt(r_1), sy.I*sy.sqrt(1 - r_1)],
				[sy.I*sy.sqrt(1 - r_1), sy.sqrt(r_1)]])
Phi = sy.Matrix([[sy.exp(sy.I * phi), 0],
				[0, 1]])
E = sy.Matrix([[E_0], [E_1]])

out = B_0 * Phi * B_1 * E
out

Matrix([
[(sqrt(r_0)*sqrt(r_1)*exp(I*\phi(t)) - sqrt(1 - r_0)*sqrt(1 - r_1))*E_{in0}(t) + (I*sqrt(r_0)*sqrt(1 - r_1)*exp(I*\phi(t)) + I*sqrt(r_1)*sqrt(1 - r_0))*E_{in1}(t)],
[(sqrt(r_0)*sqrt(r_1) - sqrt(1 - r_0)*sqrt(1 - r_1)*exp(I*\phi(t)))*E_{in1}(t) + (I*sqrt(r_0)*sqrt(1 - r_1) + I*sqrt(r_1)*sqrt(1 - r_0)*exp(I*\phi(t)))*E_{in0}(t)]])

So quite interesting, optics simply behaves different to other dynamic systems, although you could argue the delay in the heater or electro-optic modulated response is what really drives the dynamic response of the phase change. Yes, of course, because currently it's just a signal going through a block of glass with a given phase change when it is constant. We can assume the differential response of the system in time really is within $\phi (t)$. Also we are not considering the optical delay of the response yet.

Note that we're currently assuming two equal signal outputs to what came in, not necessarily the effect of a MZI. 

Now this is fully just in the optical domain, and we can begin doing more with further domains inputs, but we'll get to that. Note that the case below is for linear state space, but we'll go into nonlinear in a bit.

<img src="docs/img/linear_state_space_theory.png" />

\begin{equation}
\begin{bmatrix} E_{out0}(t) \\ E_{out1}(t) \end{bmatrix} =
\begin{bmatrix}
\left(\sqrt{r_{0}} \sqrt{r_{1}} e^{i \phi{\left(t \right)}} - \sqrt{1 - r_{0}} \sqrt{1 - r_{1}}\right) \operatorname{E_{in0}}{\left(t \right)} + \left(i \sqrt{r_{0}} \sqrt{1 - r_{1}} e^{i \phi{\left(t \right)}} + i \sqrt{r_{1}} \sqrt{1 - r_{0}}\right) \operatorname{E_{in1}}{\left(t \right)}
\\
\left(\sqrt{r_{0}} \sqrt{r_{1}} - \sqrt{1 - r_{0}} \sqrt{1 - r_{1}} e^{i \phi{\left(t \right)}}\right) \operatorname{E_{in1}}{\left(t \right)} + \left(i \sqrt{r_{0}} \sqrt{1 - r_{1}} + i \sqrt{r_{1}} \sqrt{1 - r_{0}} e^{i \phi{\left(t \right)}}\right) \operatorname{E_{in0}}{\left(t \right)} 
\end{bmatrix}
\end{equation}

The electromagnetic waves should also be the output of the system even if they're also states that have been changed throughout their dynamic response of the system. See this example https://lpsa.swarthmore.edu/Representations/SysRepSS.html on generating state space systems. Hence the output will be just:

\begin{equation} 
\textbf{y} = \begin{bmatrix} E_{out0}(t) \\ E_{out1}(t) \end{bmatrix}
\end{equation}


We know our state vector is:

\begin{equation} 
\textbf{x} = \begin{bmatrix} E_{in0}(t) \\ E_{in1}(t) \end{bmatrix}
\end{equation}

However, our input control variable $\phi$ operates nonlinearly in our system. So we're going to put the equations in a canonical nonlinear state space form, without any inputs in the system.
$R$ really gives response properties rather than control properties. But this can be modelled in different ways anyways if we wanted. In any case, now we're going to assume it's not an input vector.

\begin{equation} 
\textbf{u} = 0
\end{equation}



Now we need to generate our state matrix $\textbf{A}$. Now the reason why state space is so popular is because of how it deals with linear systems, but it can also deal with very complex nonlinear systems at controlled operating conditions.

In the case above, the MZI is a nonlinear device given interrelated states and inputs $\textbf{x}$ and $\textbf{u}$ with the $E \phi$ relationship, so they can't be separated into independent $\textbf{A}$ and $\textbf{B}$ matrices canonically, but they can if we consider the outputs of the system after it has reached steady-state equilibrium after a given change. This should be useful for most modelling situations, as you can discretize differentially the desired changes and only model the changed output in equilibrium without transient changes. This should also be less computationally expensive.

A mathemtical representation of delays in state-space systems are here: https://www.mathworks.com/help/control/ref/delayss.html and https://www.mathworks.com/help/control/ref/delayss.html . 

\begin{equation}
\textbf{y} = \textbf{C} \textbf{x} + \textbf{D} \textbf{u}
\end{equation}

\begin{equation}
\begin{bmatrix} E_{out0}(t) \\ E_{out1}(t) \end{bmatrix} = 
\begin{bmatrix}\sqrt{r_{0}} \sqrt{r_{1}} e^{i \phi{\left(t \right)}} - \sqrt{1 - r_{0}} \sqrt{1 - r_{1}} & i \sqrt{r_{0}} \sqrt{1 - r_{1}} e^{i \phi{\left(t \right)}} + i \sqrt{r_{1}} \sqrt{1 - r_{0}} \\
i\sqrt{r_{0}} \sqrt{1 - r_{1}} + i \sqrt{r_{1}} \sqrt{1 - r_{0}} e^{i \phi{\left(t \right)}} & \sqrt{r_{0}} \sqrt{r_{1}} - \sqrt{1 - r_{0}} \sqrt{1 - r_{1}} e^{i \phi{\left(t \right)}}\end{bmatrix}
 \begin{bmatrix} E_{in0}(t) \\ E_{in1}(t) \end{bmatrix}
\end{equation}

\begin{equation} 
\textbf{D} = \begin{bmatrix} 0 & 0 \\ 0 & 0 \end{bmatrix}
\end{equation}

\begin{equation} 
\textbf{C} = \begin{bmatrix}\sqrt{r_{0}} \sqrt{r_{1}} e^{i \phi{\left(t \right)}} - \sqrt{1 - r_{0}} \sqrt{1 - r_{1}} & i \sqrt{r_{0}} \sqrt{1 - r_{1}} e^{i \phi{\left(t \right)}} + i \sqrt{r_{1}} \sqrt{1 - r_{0}} \\
i\sqrt{r_{0}} \sqrt{1 - r_{1}} + i \sqrt{r_{1}} \sqrt{1 - r_{0}} e^{i \phi{\left(t \right)}} & \sqrt{r_{0}} \sqrt{r_{1}} - \sqrt{1 - r_{0}} \sqrt{1 - r_{1}} e^{i \phi{\left(t \right)}}\end{bmatrix}
\end{equation}

So in reality, for a given phi at any equilibrium moment in time, we can just input this into a state space model as a standard direct response, or include any feedback input in the input vector.

We know, from the differential equations above that, because these terms are not present.
\begin{equation} 
\dot{\textbf{x}} = \begin{bmatrix} 0 \\ 0 \end{bmatrix}
\end{equation}

Hence, we can write in the nonlinear equations:

\begin{equation}
\dot{\textbf{x}} = \textbf{0} = \textbf{A} \textbf{x} +  \textbf{B} \textbf{u}
\end{equation}

or 
\begin{equation}
\textbf{A} \textbf{x} = -  \textbf{B} \textbf{u}
\end{equation}

You could say that $\textbf{A} = \textbf{B}$ in this case, since $\textbf{B}\textbf{u} = 0$

But this does not solve the actual problem of inputting two input signals. Note we want to control those inputs, and currently we'd have to discretize to solve them or connect them separately. So state space is not applicable in this case alone, let's consider

In [ ]:
pc.ss()